In [33]:
import pymongo
from pymongo import MongoClient
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from sklearn import preprocessing
import warnings
import sys
import os
def nbipath():
    os.chdir('..')
    path = os.getcwd()+'/nbi'
    return path 

sys.path.insert(0, nbipath())
from nbi1 import *

from numpy.random import randn
from scipy.stats import anderson
from scipy.stats import normaltest
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
import scipy

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from scipy import stats


import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

warnings.filterwarnings('ignore')

In [34]:
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
 
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov

def undersample(df, keyword):
    """
    It will take a dataframe and a keyword to groupby
    :input: df: Dataframe
            Key-word: column to groupby
            
    :return: undersample_df: Datafram
    
    """
    groupby_df = df.groupby([keyword]).count()
    count = min(groupby_df[groupby_df.columns[0]].tolist())
    
    tempdf_list = []

    for index in df[keyword].unique():
        tempdf_list.append(df[df[keyword] == index].sample(count)) ## Count
    
    undersample_df = pd.concat(tempdf_list)
    
    return undersample_df

def aov(df, formula):
    """
    :input: df: DataFrame
            formula: 
    
    :return: 
            Table
    """  
    model = ols(formula, df).fit()
    aov_table = anova_lm(model, typ=2)

    eta_squared(aov_table)
    return omega_squared(aov_table)
    
def createDictionary(dataframe_col1, dataframe_col2):
    """
    returns dictionary of structure number as key and and list of attribute belonging  to structure number
    :rtype: dictionary
    
    """
    # initialize the dictionary
    dictionary = {structure_number:[] for structure_number in dataframe_col1}
    
    for structure_number, value in zip(dataframe_col1, dataframe_col2):
        dictionary[structure_number].append(value)
    return dictionary


def createDictionaryOfMeanValue(dictionary):
    """
    returns dictionary of structure number as key and and mean values in corresponding to the structure number
    :rtype: dictionary
    
    """
    dictionary_new = {}
    
    for key, value in dictionary.items():
        mean_value = np.mean(value)
        dictionary_new[key] = mean_value
        
    return dictionary_new

def createDictionaryOfFirstValue(dictionary):
    dictionary_new = {}
    
    for key, value in dictionary.items():
        value = value[0]
        dictionary_new[key] = value
        
    return dictionary_new



In [59]:
df_csv = pd.read_excel('/Users/AkshayKale/Documents/github/data/population/PEP_2018_PEPANNRES_with_ann.xls', index_col=0)
df_freeze_snow = pd.read_csv('/Users/AkshayKale/Documents/github/data/freezethaw-snowfall/freezethaw-snowfall-allstates-allyears.csv', index_col=0)
d_structureNumber_county = createDictionary(df_freeze_snow['structure_number'],df_freeze_snow['COUNTY_NAME'])
d_structureNumber_freeze_thaw = createDictionary(df_freeze_snow['structure_number'],df_freeze_snow['NO_OF_FREEZE_THAW_CYCLES'])

In [36]:
new_header = df_csv.iloc[0] #grab the first row for the header
df_csv = df_csv[1:] #take the data less the header row
df_csv.columns = new_header #set the header row as the df header

In [37]:
dict_county_population = {}
for county, population in zip(df_csv['Geography'], df_csv['Population Estimate (as of July 1) - 2018']):
    county_name, state = county.split(',')
    dict_county_population[county_name] = population

In [38]:
csv_file = "/Users/AkshayKale/Documents/github/data/nbi/06-20-19-thesis-dataset-without-outliers_allstates"
df_wo = pd.read_csv(csv_file, index_col = 0)

In [39]:
d_structureNumber_county = createDictionary(df_freeze_snow['structure_number'], df_freeze_snow['COUNTY_NAME'])

In [60]:
d_freeze_thaw = createDictionaryOfMeanValue(d_structureNumber_freeze_thaw)

In [41]:
d_struct_county = {key: value[0] for key, value in d_structureNumber_county.items()}

In [46]:
d_structure_population = {}
for structure_number, county in d_struct_county.items():
    population = dict_county_population.get(county)
    d_structure_population[structure_number] = population

In [48]:
df_wo = df_wo[df_wo['No Change structure length'] == True]
df_wo = df_wo[~df_wo['Structure Length'].isin([-1,0, ])]
df_wo = df_wo[df_wo['Structure Length']>6]

In [62]:
mapping = df_wo
mapping['SN'] = [structureNumber[:-2] for structureNumber in mapping['Structure Number']]
mapping['No. of Freeze Thaw'] = mapping['SN'].map(d_freeze_thaw)
mapping['county'] = mapping['SN'].map(d_structureNumber_county)
mapping['population'] = mapping['SN'].map(d_freeze_thaw)
mapping = mapping.dropna()
mapping['county'] = [county[0] for county in mapping['county']]
mapping['label'] = pd.qcut(mapping['No. of Freeze Thaw'],[0, .05, .95, 1.], labels = ['Very Low','Medium','Very High'])

In [ ]:
def makefloat(items):
    """
       Return a list of float from a list of strings.
    """
    items_int = []
    for item in items:
        try:
            items_int.append(float(item))
        except:
            pass
    return items_int
            
            
def getNonZeroValue(items):
    """
       Returns first non-zero value from the string.
    """
    for item in items:
        if item != 0.0:
            return item
        

def parseCoordinate(df_column): 
    """
       Returns a list of coordinatenate from list of lists of repeated or null coordinates.
    """
    coordinate = []
    for coords in df_column:
        list_coords = coords[1:-1].split(',')
        list_coords = makefloat(list_coords)
        coords = getNonZeroValue(list_coords)     
        coordinate.append(coords)
    return coordinate

mapping['longitude'] = parseCoordinate(mapping['Longitude'])
mapping['latitude'] = parseCoordinate(mapping['Latitude'])

In [65]:
NE_clean['freezethaw'] = pd.qcut(NE_clean['No. of Freeze Thaw'],[0, .05, .95, 1.], labels = ['Very Low','Medium','Very High'])

df_anova = NE_clean[['Baseline Difference Score', 'ADT Category','ADTT Category','Avg. Daily Precipitation', 'State Code',
                       'Material','Structure Type', 'Maintainer','freezethaw', 'No. of Freeze Thaw', 'Average ADT', 'Longitude', 'Latitude']]


df_anova = df_anova.rename(index=str, columns={"Baseline Difference Score": "score", \
                                               "ADT Category": "adt", \
                                               "ADTT Category": "adtt",\
                                               "Avg. Daily Precipitation": "precp", \
                                               "State Code":"region", \
                                               "Material":"material",\
                                               "Structure Type":"structuretype",\
                                               "Maintainer":"maintainer",\
                                               'freezethaw':'freezethaw',\
                                               'No. of Freeze Thaw':'numfreezethaw',
                                               'Average ADT':'aadtt',
                                               'Longitude':'longitude',
                                               'Latitude':'latitude'
                                              })

,Structure Number,Age,ADT Type,ADTT Type,Superstructure,Avg. Daily Precipitation,State Code,Average Daily Traffic,Owner,Maintainer,...,Baseline Difference Score,Scaled Baseline Difference Score,SN,No. of Freeze Thaw,Population,county,population,label,longitude,latitude
3,000000000000001_1,"[33, 34, 35, 36]","['Heavy', 'Heavy', 'Heavy', 'Heavy']","['Light', 'Light', 'Moderate', 'Moderate']","['7', '7', '6', '6']",3.10,51,"[27400, 27400, 27400, 27400]",66,66,...,-0.154910,-0.019313,000000000000001,96.789474,20348.0,Adams County,96.789474,Medium,-77.118333,38.928333
5,000000000000002_1,"[58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 6...","['Very Light', 'Light', 'Light', 'Light', 'Lig...","['Light', 'Light', 'Light', 'Light', 'Light', ...","['6', '6', '6', '6', '8', '8', '8', '8', '8', ...",0.87,35,"[98, 100, 100, 111, 111, 54, 56, 60, 60, 64, 6...",1,1,...,0.600322,0.754025,000000000000002,108.539474,20348.0,Adams County,108.539474,Medium,-107.291667,34.940000
7,000000000000003_1,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['Light', 'Light', 'Moderate', 'Moderate', 'Mo...","['4', '4', '4', '6', '6', '6', '6', '7', '7', ...",3.10,51,"[10850, 10850, 10850, 10850, 10850, 13000, 130...",27,27,...,-0.274255,-0.141518,000000000000003,89.289474,20348.0,Adams County,89.289474,Medium,-77.121667,38.926667
13,000000000000007_2,"[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 4...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['Moderate', 'Moderate', 'Moderate', 'Moderate...","['8', '8', '8', '8', '6', '6', '6', '6', '6', ...",3.10,51,"[10200, 10200, 10200, 10200, 10200, 10200, 102...",1,1,...,-0.399688,-0.269959,000000000000007,75.157895,10362.0,Alcona County,75.157895,Medium,-77.083333,38.886667
20,000000000000010_1,"[16, 17, 18, 19, 20, 21, 22]","['Moderate', 'Moderate', 'Moderate', 'Moderate...","['Light', 'Light', 'Light', 'Light', 'Light', ...","['8', '8', '8', '7', '7', '7', '7']",2.06,26,"[1450, 1450, 1450, 1450, 1450, 1450, 1450]",2,2,...,0.163684,0.306920,000000000000010,93.710526,10362.0,Alcona County,93.710526,Medium,-83.448089,44.590383
23,000000000000012_1,"[17, 18, 19, 20, 21, 22, 23]","['Light', 'Light', 'Light', 'Light', 'Light', ...","['Light', 'Light', 'Light', 'Light', 'Light', ...","['8', '8', '8', '7', '7', '7', '7']",2.06,26,"[211, 211, 211, 211, 211, 211, 211]",2,2,...,0.210114,0.354463,000000000000012,75.157895,10362.0,Alcona County,75.157895,Medium,-83.335767,44.750919
24,000000000000012_2,"[21, 22, 23, 24, 25, 26, 27]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['8', '8', '8', '8', '8', '7', '7']",3.27,42,"[10284, 10459, 10459, 9551, 9180, 9298, 9698]",1,1,...,0.674357,0.829835,000000000000012,75.157895,10362.0,Alcona County,75.157895,Medium,-77.208311,39.792250
28,000000000000014_2,"[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['7', '7', '7', '7', '7', '7', '7', '6', '6', ...",3.10,51,"[17000, 19200, 19200, 19200, 19200, 19200, 192...",2,2,...,-0.661148,-0.537686,000000000000014,88.166667,10362.0,Alcona County,88.166667,Medium,-77.095000,38.845000
29,000000000000014_3,"[21, 22, 23, 24, 25, 26, 27]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['8', '8', '8', '8', '8', '7', '7']",3.27,42,"[11348, 11065, 10912, 10912, 10867, 10822, 12275]",1,1,...,0.674357,0.829835,000000000000014,88.166667,10362.0,Alcona County,88.166667,Medium,-77.198664,39.824611
32,000000000000016_1,"[31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 4...","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","['Moderate', 'Moderate', 'Moderate', 'Moderate...","['8', '8', '7', '7', '6', '6', '6', '6', '6', ...",3.10,51,"[16000, 16000, 15500, 15500, 15500, 15500, 155...",2,2,...,-0.500264,-0.372945,000000000000016,67.855263,10362.0,Alcona County,67.855263,Mediu

In [ ]:
df_anova_mapping = df_anova[['numsnowfall', 'longitude', 'latitude', 'score', 'snowfall']]
df_anova_mapping.to_csv('../data/nbi/mapping-snowfall.csv')